In [128]:
import tensorflow as tf 
import matplotlib as mpl
import matplotlib.pyplot as plt

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [157]:
from pandas import read_csv
from datetime import datetime
# load data
def parser(x):
	return datetime.strptime(x, '%Y-%m-%d')
dataset = read_csv('/content/drive/MyDrive/VDT2022/mini-project/stroke_recovery_dataset/protocol/sb01.csv',  parse_dates = [1], index_col=0, date_parser=parser)

dataset = dataset.set_index('Date')
dataset.columns = ['counts', 'speed']
# summarize first 5 rows
print(dataset.head(5))

            counts   speed
Date                      
2020-01-01   394.0  1.1506
2020-01-02   397.0  1.1504
2020-01-03   382.0  1.1563
2020-01-04   395.0  1.1561
2020-01-05   394.0  1.1540


In [158]:
plt.plot(dataset.counts)

In [173]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [174]:
# load dataset
def parser(x):
	return datetime.strptime(x, '%Y-%m-%d')
dataset = read_csv('/content/drive/MyDrive/VDT2022/mini-project/stroke_recovery_dataset/protocol/sb01.csv',  parse_dates = [1], index_col=0, date_parser=parser)

dataset = dataset.set_index('Date')
dataset.columns = ['counts', 'speed']

cols = ['counts', 'speed']
dataset = dataset.loc[dataset.index, cols]
dataset = dataset.interpolate()
dataset.head(5)

,counts,speed
Date,,
2020-01-01,394.0,1.1506
2020-01-02,397.0,1.1504
2020-01-03,382.0,1.1563
2020-01-04,395.0,1.1561
2020-01-05,394.0,1.1540


In [183]:
# counts_trend_data = read_csv('/content/counts_trend.csv')
# speed_trend_data = read_csv('/content/speed_trend.csv')

In [184]:
# dataset.counts[0:274] = counts_trend_data.values[:,1]
# dataset.speed[0:274] = speed_trend_data.values[:,1]

In [ ]:
# dataset.head(5)

In [186]:
values = dataset.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [187]:
# specify the number of lag days
n_hours = 3
n_features = 2
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train = 274
train = values[:n_train, :]
test = values[n_train:, :]

(363, 8)


In [188]:
# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, 6:8]
test_X, test_y = test[:, :n_obs], test[:, 6:8]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(274, 6) 274 (274, 2)
(274, 3, 2) (274, 2) (89, 3, 2) (89, 2)


Epoch 1/310
3/3 - 4s - loss: 0.4071 - val_loss: 0.8259 - 4s/epoch - 1s/step
Epoch 2/310
3/3 - 0s - loss: 0.3784 - val_loss: 0.7784 - 37ms/epoch - 12ms/step
Epoch 3/310
3/3 - 0s - loss: 0.3499 - val_loss: 0.7299 - 36ms/epoch - 12ms/step
Epoch 4/310
3/3 - 0s - loss: 0.3213 - val_loss: 0.6804 - 37ms/epoch - 12ms/step
Epoch 5/310
3/3 - 0s - loss: 0.2931 - val_loss: 0.6292 - 39ms/epoch - 13ms/step
Epoch 6/310
3/3 - 0s - loss: 0.2640 - val_loss: 0.5752 - 44ms/epoch - 15ms/step
Epoch 7/310
3/3 - 0s - loss: 0.2335 - val_loss: 0.5173 - 40ms/epoch - 13ms/step
Epoch 8/310
3/3 - 0s - loss: 0.2013 - val_loss: 0.4545 - 37ms/epoch - 12ms/step
Epoch 9/310
3/3 - 0s - loss: 0.1668 - val_loss: 0.3857 - 41ms/epoch - 14ms/step
Epoch 10/310
3/3 - 0s - loss: 0.1306 - val_loss: 0.3110 - 41ms/epoch - 14ms/step
Epoch 11/310
3/3 - 0s - loss: 0.0986 - val_loss: 0.2319 - 43ms/epoch - 14ms/step
Epoch 12/310
3/3 - 0s - loss: 0.0699 - val_loss: 0.1510 - 43ms/epoch - 14ms/step
Epoch 13/310
3/3 - 0s - loss: 0.0615 - va

In [240]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(2))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=310, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [241]:
# make a prediction counts
yhat = model.predict(test_X)

inv_yhat = scaler.inverse_transform(yhat)
inv_y = scaler.inverse_transform(test_y)

In [242]:
counts_y = inv_y[:,0]
counts_yhat = inv_yhat[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(counts_y, counts_yhat))
print('Test RMSE: %.6f' % rmse)
# calculate MAE
mae = mean_absolute_error(counts_y, counts_yhat)
print('Test MAE: %.6f' % mae)
# calculate MAPE
mape = mean_absolute_percentage_error(counts_y, counts_yhat)
print('Test MAPE: %.6f' % mape)

Test RMSE: 2.198180
Test MAE: 1.784057
Test MAPE: 0.003897


In [243]:
speed_y = inv_y[:,1]
speed_yhat = inv_yhat[:,1]
# calculate RMSE
rmse = sqrt(mean_squared_error(speed_y, speed_yhat))
print('Test RMSE: %.6f' % rmse)
# calculate MAE
mae = mean_absolute_error(speed_y, speed_yhat)
print('Test MAE: %.6f' % mae)
# calculate MAPE
mape = mean_absolute_percentage_error(speed_y, speed_yhat)
print('Test MAPE: %.6f' % mape)

Test RMSE: 0.002567
Test MAE: 0.002038
Test MAPE: 0.001656
